In [3]:
import pydicom
import matplotlib.pyplot as plt
import os
import nibabel as nib
import numpy as np
import pandas as pd
import cv2
import pickle

In [4]:
#path to the dicom input folder
dicom_folder_path = '/Users/arjunmoorthy/Desktop/targeted_bx_dicoms/targeted_bx_slices/'

In [5]:
# Initialize an empty list to store the dictionaries containing the DICOM paths and data
dicoms = []

# Walk through all subdirectories and files in dicom_folder_path
for dirpath, dirnames, filenames in os.walk(dicom_folder_path):
    for filename in filenames:
        # Construct the full path for each file
        file_path = os.path.join(dirpath, filename)
        # Attempt to read the DICOM file
        try:
            dicom = pydicom.dcmread(file_path, force=True)
            # Check if 'InstanceNumber' is in the DICOM tags
            if 'InstanceNumber' in dicom:
                # Append the dictionary with path and dicom data to the list
                dicoms.append({'path': file_path, 'dicom': dicom})
        except Exception as e:
            # Handle cases where the file might not be a readable DICOM file
            print(f"Error reading {file_path}: {e}")

In [7]:
# Sort the dicom based on the instance number
dicoms = sorted(dicoms, key=lambda x: x['dicom'].InstanceNumber)

In [8]:
# Function to remove duplicate slices
def remove_duplicate_slices(dicoms):
    dicoms_dict = {}
    filtered_dicoms = []
    if 'ImageOrientationPatient' not in dicoms[0]['dicom']:
        return dicoms
    for dicom in dicoms:
        if tuple(dicom['dicom'].ImagePositionPatient) not in dicoms_dict:
            dicoms_dict[tuple(dicom['dicom'].ImagePositionPatient)] = dicom['dicom']
            filtered_dicoms.append(dicom)
        else:
            if np.array_equal(dicom['dicom'].pixel_array, dicoms_dict[tuple(dicom['dicom'].ImagePositionPatient)].pixel_array): continue
            else:
                filtered_dicoms.append(dicom)
    return filtered_dicoms

In [9]:
print('before removing duplicates: ', len(dicoms))
sorted_dicoms = remove_duplicate_slices(dicoms)
print('after removing duplicates', len(dicoms))

before removing duplicates:  30
after removing duplicates 30


In [11]:
# Create a new list in which the dicom image path, patient position and their slice locations are saved
data_dicom = []
for d in sorted_dicoms:
    data_dicom.append((d['path'],d['dicom']['ImagePositionPatient'].value,float(d['dicom']['ImagePositionPatient'].value[-1]),int(d['dicom']['InstanceNumber'].value)))
data_dicom

[('/Users/arjunmoorthy/Desktop/targeted_bx_dicoms/targeted_bx_slices/Prostate-MRI-US-Biopsy-0001/1-20.dcm',
  [-107.93213844258, -66.044486999928, -46.381074011326],
  -46.381074011326,
  20),
 ('/Users/arjunmoorthy/Desktop/targeted_bx_dicoms/targeted_bx_slices/Prostate-MRI-US-Biopsy-0001/1-23.dcm',
  [-107.93213844258, -66.044486999928, -41.881074011326],
  -41.881074011326,
  23),
 ('/Users/arjunmoorthy/Desktop/targeted_bx_dicoms/targeted_bx_slices/Prostate-MRI-US-Biopsy-0001/1-24.dcm',
  [-107.93213844258, -66.044486999928, -40.381074011326],
  -40.381074011326,
  24),
 ('/Users/arjunmoorthy/Desktop/targeted_bx_dicoms/targeted_bx_slices/Prostate-MRI-US-Biopsy-0012/1-26.dcm',
  [-85, -44.334438323975, -20.243975907564],
  -20.243975907564,
  26),
 ('/Users/arjunmoorthy/Desktop/targeted_bx_dicoms/targeted_bx_slices/Prostate-MRI-US-Biopsy-0012/1-27.dcm',
  [-85, -44.334438323975, -18.743975907564],
  -18.743975907564,
  27),
 ('/Users/arjunmoorthy/Desktop/targeted_bx_dicoms/targeted_bx

In [ ]:
with open('patientxydictinfo.pkl', 'rb') as file:
    # Load the object from the file
    my_object = pickle.load(file)